In [3]:
import sys

import numpy as np
import pandas as pd
import seaborn as sns

In [4]:
sys.path.insert(0, '../modules/')
import cleaning as cln
import feature_eng as feng

In [5]:
#Import data
store = pd.read_csv('../data/store.csv', index_col='Store')
train = pd.read_csv('../data/train.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Merging and cleaning data

In [8]:
#Merge data
df = cln.merge(train, store)
df.head()

,Date,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,2013-01-01,1115.0,2.0,0.0,0.0,0.0,0.0,a,1.0,d,c,5350.0,NaN,NaN,1.0,22.0,2012.0,"Mar,Jun,Sept,Dec"
1,2013-01-01,379.0,2.0,0.0,0.0,0.0,0.0,a,1.0,d,a,6630.0,NaN,NaN,0.0,NaN,NaN,NaN
2,2013-01-01,378.0,2.0,0.0,0.0,0.0,0.0,a,1.0,a,c,2140.0,8.0,2012.0,0.0,NaN,NaN,NaN
3,2013-01-01,377.0,2.0,0.0,0.0,0.0,0.0,a,1.0,a,c,100.0,6.0,2010.0,1.0,18.0,2010.0,"Feb,May,Aug,Nov"
4,2013-01-01,376.0,2.0,0.0,0.0,0.0,0.0,a,1.0,a,a,160.0,8.0,2012.0,0.0,NaN,NaN,NaN


In [9]:
#Drop customers column
df = cln.drop_column(df)

In [11]:
#Clean the StateHoliday column
df = cln.clean_column_values(df)

In [12]:
#Remove null and zero values for Sales
df = cln.clean_targets(df)

In [13]:
#Use the cleaning function
df = cln.rough_features_cleaning(df)

Total number of rows before cleaning:  515849
Total number of rows after cleaning:  425689


In [14]:
df.head()

,Date,Store,DayOfWeek,Sales,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2
27,2013-01-01,353.0,2.0,3139.0,1.0,0.0,a,1.0,b,b,900.0,1.0
115,2013-01-01,335.0,2.0,2401.0,1.0,0.0,a,1.0,b,a,90.0,1.0
147,2013-01-01,512.0,2.0,2646.0,1.0,0.0,a,1.0,b,b,590.0,1.0
162,2013-01-01,494.0,2.0,3113.0,1.0,0.0,a,1.0,b,a,1260.0,0.0
199,2013-01-01,530.0,2.0,2907.0,1.0,0.0,a,1.0,a,c,18160.0,0.0


In [15]:
df.isna().any()

Date                   False
Store                  False
DayOfWeek              False
Sales                  False
Open                   False
Promo                  False
StateHoliday           False
SchoolHoliday          False
StoreType              False
Assortment             False
CompetitionDistance    False
Promo2                 False
dtype: bool

### Feature engineering

In [16]:
#Add new dates features
df = feng.dates_features(df)

In [17]:
df.head()

,Date,Store,Sales,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,month,day_of_week,day_of_month,is_monday,is_saturday
27,2013-01-01,353.0,3139.0,1.0,0.0,a,1.0,b,b,900.0,1.0,1,1,1,0,0
115,2013-01-01,335.0,2401.0,1.0,0.0,a,1.0,b,a,90.0,1.0,1,1,1,0,0
147,2013-01-01,512.0,2646.0,1.0,0.0,a,1.0,b,b,590.0,1.0,1,1,1,0,0
162,2013-01-01,494.0,3113.0,1.0,0.0,a,1.0,b,a,1260.0,0.0,1,1,1,0,0
199,2013-01-01,530.0,2907.0,1.0,0.0,a,1.0,a,c,18160.0,0.0,1,1,1,0,0


In [18]:
#Splitting data in train and test
df_train, df_test = feng.date_split_train_test(df, '2014-05-01')

In [20]:
#One hot encoding of StateHoliday, StoreType, Assortment for train
df_train = feng.one_hot_encoding(df_train, 'StateHoliday')
df_train = feng.one_hot_encoding(df_train, 'StoreType')
df_train = feng.one_hot_encoding(df_train, 'Assortment')

In [22]:
#One hot encoding of StateHoliday, StoreType, Assortment for test
df_test = feng.one_hot_encoding(df_test, 'StateHoliday')
df_test = feng.one_hot_encoding(df_test, 'StoreType')
df_test = feng.one_hot_encoding(df_test, 'Assortment')

In [24]:
#Add mean encoding for the Store id for train
df_train, dict_store_values = feng.mean_encoding(df_train, 'Store')

In [25]:
df_train.head()

,Date,Store,Sales,Open,Promo,SchoolHoliday,CompetitionDistance,Promo2,month,day_of_week,...,StateHoliday _b,StateHoliday _c,StoreType _a,StoreType _b,StoreType _c,StoreType _d,Assortment _a,Assortment _b,Assortment _c,Store_mean_encoded
27,2013-01-01,353.0,3139.0,1.0,0.0,1.0,900.0,1.0,1,1,...,0,0,0,1,0,0,0,1,0,4975.040107
115,2013-01-01,335.0,2401.0,1.0,0.0,1.0,90.0,1.0,1,1,...,0,0,0,1,0,0,1,0,0,13029.339286
147,2013-01-01,512.0,2646.0,1.0,0.0,1.0,590.0,1.0,1,1,...,0,0,0,1,0,0,0,1,0,5075.170732
162,2013-01-01,494.0,3113.0,1.0,0.0,1.0,1260.0,0.0,1,1,...,0,0,0,1,0,0,1,0,0,7478.446429
199,2013-01-01,530.0,2907.0,1.0,0.0,1.0,18160.0,0.0,1,1,...,0,0,1,0,0,0,0,0,1,4178.085561


In [32]:
#Apply same transformation to test with the values from train
df_test.loc[:, 'Store_mean_encoded'] = df_test.loc[:, 'Store'].replace(to_replace=dict_values)

540889    13029.339286
540921     5075.170732
540934     7478.446429
541068     7096.483544
541169     7798.825000
Name: Store, dtype: float64